# The Classiq Workflow

This notebook introduces the typical workflow with Classiq:
- **Designing a quantum algorithm** using the Qmod language and it's accompanied function library.
- **Synthesizing the algorithm** into a concrete circuit implementation.
- **Executing the algorithm** on a chosen simulator or quantum harware.
- **Post-processing** the results.

In [1]:
from classiq import *